## **HDBnet - Data Cleaning**

In this notebook we will perform data cleaning and feature engineering in preparation of regressive modelling. The objective is to achieve an MAE of less than 20k SGD.

In [1]:
import pandas as pd
import concurrent.futures
import requests
import time
import os
import random
import pickle

Download Housing Data

In [2]:
import os
from zipfile import ZipFile 

filename = 'download.zip'

if not os.path.isfile(filename):
    # download the latest hdb resale price index
    !wget "https://data.gov.sg/dataset/7a339d20-3c57-4b11-a695-9348adfd7614/download"
    os.rename('download',filename)

    with ZipFile(filename, 'r') as zipped: 
        zipped.printdir() 
        zipped.extractall()


###**Ingestion**
The HDB resale price transaction records provides data on all resale flat transactions from 1990 till present. The HDB resale transaction data from data.gov.sg comes in a zip file containing five csv files. The 30 year records are split over the five files, and covers 1990-2000, 2000-2012, 2012-2014, 2015-2016 and 2017-2020. There are schema differences between the files and must be accounted for prior to concatenation. To maintain the brevity and relevance of the dataset, we will only utilise data from four files which covers the past 20 years from 2000-2020. 

In [3]:
housing_data_2017 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [4]:
display(housing_data_2017.head())

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


### **Data Cleaning**
We can observe some pecularities in the datasets.
* 2017: The remaining lease column lists the years and months remaining at the point of sale
* 2012: Does not contain remaining lease column. Will need to recreate remaining lease using lease_commence_date + 99 - year of sale
* 2000: Does not contain remaining lease column. Will need to recreate remaining lease using lease_commence_date + 99 - year of sale

Our plan is to align the 2000, 2012 and 2017 datasets to the 2015 dataset. Aligning the features above will enable concatenation of the four datasets in to a full HDB resale dataset covering 2000-2020 transaction data.

There are also certain features that need to be cleaned in the combined dataset to ease modelling in the later stages.

<br>

**storey_range**

The storey_range column also needs to be cleaned. Some units are assigned '01 TO 05', while others are '01 TO 03'. The unit levels are not always categorized in groups of 3 levels. We will need to record the min and max levels, and do a random sample from there.

In [5]:
def extract_year_month(df):
    df['year'] = df['month'].str[0:4].astype(int)
    df['month'] = df['month'].str[-2:].astype(int)
    return(df)

# Prepare year and month for all datasets
housing_data_2017 = extract_year_month(housing_data_2017)

# Extract remaining lease year from 2017+ dataset
housing_data_2017['remaining_lease'] = housing_data_2017['remaining_lease'].str[:2].astype('int')
housing_data_raw = housing_data_2017

# Random sample from storey range
housing_data_raw['storey'] = housing_data_raw['storey_range'].apply(lambda x: random.choice(range(int(x[0:2]), int(x[-2:]))))

# Drop rows earlier than 2021
housing_data_raw = housing_data_raw[housing_data_raw['year']>=2021]

In [6]:
display(housing_data_raw)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,storey
87589,1,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,01 TO 03,45.0,Improved,1986,64,211000.0,2021,1
87590,1,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,64,225000.0,2021,7
87591,1,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,04 TO 06,68.0,New Generation,1981,59,260000.0,2021,5
87592,1,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,58,265000.0,2021,4
87593,1,ANG MO KIO,3 ROOM,561,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1980,58,265000.0,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149798,3,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,64,800000.0,2023,11
149799,3,YISHUN,EXECUTIVE,356,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,64,800000.0,2023,1
149800,3,YISHUN,EXECUTIVE,387,YISHUN RING RD,04 TO 06,142.0,Apartment,1988,64,770000.0,2023,4
149801,3,YISHUN,EXECUTIVE,792,YISHUN RING RD,10 TO 12,146.0,Maisonette,1987,63,818000.0,2023,11



###**Feature Engineering**


There's some cleaning that needs to be done. We only keep samples from the past 2 years. We'll need to split the 'quarter' column into year and quarter to facilitate the join with the main dataset later.

In [7]:
# housing_data_raw['quarter'] = housing_data_raw['month'].div(4).add(1).astype(int)
# housing_data_raw['index'] = housing_data_raw['index'].fillna(1) #use 1 for latest quarter

#cast numerical features to float
housing_data_raw['storey'] = housing_data_raw['storey'].astype('float')
housing_data_raw['remaining_lease'] = housing_data_raw['remaining_lease'].astype('float')

#create psm feature
housing_data_raw['psm'] = housing_data_raw['resale_price'].div(housing_data_raw['floor_area_sqm'])

#create address feature
housing_data_raw['address'] = housing_data_raw['block'] + " " + housing_data_raw['street_name']

#drop unused features
housing_data_raw = housing_data_raw.drop(columns=['month','lease_commence_date','storey_range'])

/var/folders/86/bw99tfm9017ggqrmtr6qd_600000gn/T/ipykernel_59411/531528638.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_data_raw['storey'] = housing_data_raw['storey'].astype('float')
/var/folders/86/bw99tfm9017ggqrmtr6qd_600000gn/T/ipykernel_59411/531528638.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_data_raw['remaining_lease'] = housing_data_raw['remaining_lease'].astype('float')
/var/folders/86/bw99tfm9017ggqrmtr6qd_600000gn/T/ipykernel_59411/531528638.py:9: SettingWithCopy

In [8]:
display(housing_data_raw.head())
print(housing_data_raw.info())

,town,flat_type,block,street_name,floor_area_sqm,flat_model,remaining_lease,resale_price,year,storey,psm,address
87589,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,45.0,Improved,64.0,211000.0,2021,1.0,4688.888889,170 ANG MO KIO AVE 4
87590,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,45.0,Improved,64.0,225000.0,2021,7.0,5000.000000,170 ANG MO KIO AVE 4
87591,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,68.0,New Generation,59.0,260000.0,2021,5.0,3823.529412,331 ANG MO KIO AVE 1
87592,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,68.0,New Generation,58.0,265000.0,2021,4.0,3897.058824,534 ANG MO KIO AVE 10
87593,ANG MO KIO,3 ROOM,561,ANG MO KIO AVE 10,68.0,New Generation,58.0,265000.0,2021,1.0,3897.058824,561 ANG MO KIO AVE 10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 62214 entries, 87589 to 149802
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   town             62214 non-null  object 
 1   flat_type        62214 non-null  object 
 2   block            62214 non-null  object 
 3   street_name      62214 non-null  object 
 4   floor_area_sqm   62214 non-null  float64
 5   flat_model       62214 non-null  object 
 6   remaining_lease  62214 non-null  float64
 7   resale_price     62214 non-null  float64
 8   year             62214 non-null  int64  
 9   storey           62214 non-null  float64
 10  psm              62214 non-null  float64
 11  address          62214 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 6.2+ MB
None


**Integrating Distance to Amentities & CBD**

It is intuitive that the distance to nearby amenities is a significant factor in the pricing of HDB flats. The nearer units are to MRTs/LRTs and malls, the more convenient they are and thus command a higher value. As a baseline, we can tap on the search API offered by the OneMaps system to obtain geocoordinates of each HDB block, as well as the coordinates of MRTs and malls. A feature we could engineer is distance_to_mrt and distance_to_mall, which should show the distance (or walking time) to the nearest MRT/Mall.

Flats nearer to CBD also command a higher price due to its convenience. We can include distance_to_cbd as a feature by calculating the haversine distance from the block to Raffles Place MRT

**Converting Blocks to Coordinates**

We will utilise the OneMaps Search API to convert the addressses to lat long coordinates. The Search API comes in the form of a GET request, with no requirement for token authentication. Hence, we should be able to call it without easily hitting the rate limiter.



In [9]:
import urllib
addresses = housing_data_raw['address'].unique()
print(len(addresses))

9142


We create a scraping function that queries the OneMaps Search API to derive the geocoordinates of each HDB Block. We then save the results in a pickle file so that we can immediately restore the variables if we rerun the program.

In [10]:
def getcoordinates(addresses):

    """Searches for address's geocoordinates (lat, long) on OneMap's Search API

    Parameters:
    addresses (list(string)): list of addresses to search 

    Returns:
    input (list(string)): list of input addresses by order of request completion
    latitudes (list(string)): list of latitudes by order of address in input
    longitudes (list(string)): list of longitudes by order of address in input
    not_found (list(string)): list of addresses with no results in OneMap Search API

    """

    input = []
    latitudes = []
    longitudes = []
    not_found = []
    CONNECTIONS = 100
    TIMEOUT = 5

    time1 = time.time()

    queries = [urllib.parse.quote(address).replace(".","") for address in addresses] #special char replacement for url

    url1 = "https://developers.onemap.sg/commonapi/search?searchVal="
    url2 = "&returnGeom=Y&getAddrDetails=N&pageNum=1"
    urls = [ url1+query+url2 for query in queries]

    address_url_map = dict(zip(urls, addresses)) 

    def load_url(url,timeout):
        r = requests.get(url)
        return r.json()

    print(f'Fetch initiated for {len(urls)} urls.')
    with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
        future_to_url = dict((executor.submit(load_url, url, TIMEOUT), address_url_map[url]) for url in urls)
        for future in concurrent.futures.as_completed(future_to_url):
            try:
                data = future.result()
            except Exception as exc:
                data = str(type(exc))
            finally:
                if(data['found']==0): #store address of notfounds in separate list
                    not_found.append(future_to_url[future])
                    print(future_to_url[future])
                    print(data)
                else: #only append those found in input,latitudes, longitudes lists
                    input.append(future_to_url[future]) 
                    latitudes.append(data['results'][0]['LATITUDE'])
                    longitudes.append(data['results'][0]['LONGITUDE'])

    time2 = time.time()
    print(f"Search took {time2-time1:.2f} s")
    print(f'Fetch completed for {len(urls)} urls, {len(input)} urls were located, {len(not_found)} urls could not be found.')
    return input, latitudes, longitudes, not_found

In [11]:
if not os.path.isfile('geocoordinates.pickle'):
    input, latitudes, longitudes, not_found = getcoordinates(addresses)
    with open('geocoordinates.pickle', 'wb') as f:
        pickle.dump([input,latitudes,longitudes,not_found], f)
else:
    with open('geocoordinates.pickle', 'rb') as f:
        input, latitudes, longitudes, not_found = pickle.load(f)

A search on the addresses in the not_found list reveal that these HDB blocks have been demolished some time ago. We can consider dropping these samples from the dataset.

In [12]:
print(input)
print(latitudes)
print(longitudes)
print(not_found)

['624 ANG MO KIO AVE 4', '82 BEDOK NTH RD', '666 JLN DAMAI', '471 ANG MO KIO AVE 10', '441 ANG MO KIO AVE 10', '121 ANG MO KIO AVE 3', '513 BEDOK NTH AVE 2', '711 BEDOK RESERVOIR RD', '457 ANG MO KIO AVE 10', '348 ANG MO KIO AVE 3', '345 ANG MO KIO AVE 3', '107 BEDOK NTH RD', '104 BEDOK RESERVOIR RD', '126 ANG MO KIO AVE 3', '27 NEW UPP CHANGI RD', '710 BEDOK RESERVOIR RD', '181 ANG MO KIO AVE 5', '204 ANG MO KIO AVE 3', '124 BEDOK NTH RD', '621 BEDOK RESERVOIR RD', '538 ANG MO KIO AVE 5', '510 BEDOK NTH ST 3', '714 BEDOK RESERVOIR RD', '232 ANG MO KIO AVE 3', '576 ANG MO KIO AVE 10', '614 BEDOK RESERVOIR RD', '133 BEDOK NTH AVE 3', '448 ANG MO KIO AVE 10', '628 BEDOK RESERVOIR RD', '428 BEDOK NTH RD', '630 ANG MO KIO AVE 4', '509 ANG MO KIO AVE 8', '527 BEDOK NTH ST 3', '411 BEDOK NTH AVE 2', '601 ANG MO KIO AVE 5', '40 CHAI CHEE AVE', '159 BEDOK STH AVE 3', '413 ANG MO KIO AVE 10', '180 ANG MO KIO AVE 5', '502 BEDOK NTH ST 3', '523 BEDOK NTH ST 3', '129 ANG MO KIO AVE 3', '520 BEDOK 

In [13]:
df_geo = pd.DataFrame()
df_geo['address'] = pd.Series(input)
df_geo['latitude'] = pd.Series(latitudes)
df_geo['longitude'] = pd.Series(longitudes)
display(df_geo.head())
print(df_geo.info())

,address,latitude,longitude
0,624 ANG MO KIO AVE 4,1.38156171229824,103.84000226873
1,82 BEDOK NTH RD,1.32953680475668,103.940406562732
2,666 JLN DAMAI,1.3335654358321,103.908102521069
3,471 ANG MO KIO AVE 10,1.36346600647245,103.856702918462
4,441 ANG MO KIO AVE 10,1.36605027829032,103.854168309853


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9142 entries, 0 to 9141
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   address    9142 non-null   object
 1   latitude   9142 non-null   object
 2   longitude  9142 non-null   object
dtypes: object(3)
memory usage: 214.4+ KB
None


**Load MRT Station & Mall datasets**

Now that we have the geocoordinates for the 9130 valid addresses (sans 78 demolished/missing blocks), we want to obtain a list of mrt stations and malls in Singapore.

In [14]:

mrt_data = pd.read_csv('mrt_station_names_and_commencement.csv')
mall_data = pd.read_csv('shopping_mall_names_and_commencement.csv')
display(mrt_data.head())
print(mrt_data.info())
display(mall_data.head())
print(mall_data.info())

,stn_code,mrt_station_english,mrt_line_english,line_commencement
0,NS1,Jurong East,North South Line,1990
1,NS2,Bukit Batok,North South Line,1990
2,NS3,Bukit Gombak,North South Line,1990
3,NS4,Choa Chu Kang,North South Line,1990
4,NS5,Yew Tee,North South Line,1996


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   stn_code             188 non-null    object
 1   mrt_station_english  188 non-null    object
 2   mrt_line_english     188 non-null    object
 3   line_commencement    188 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 6.0+ KB
None


,mall_name,mall_commencement
0,100 AM,2019
1,600 @ Toa Payoh,1972
2,Anchorpoint,1997
3,Beauty World Centre,1984
4,Beauty World Plaza,1984


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   mall_name          125 non-null    object
 1   mall_commencement  125 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ KB
None


**Convert MRT stations & Malls to Coordinates**

Again we use the getcoordinates() function to translate the MRT stations and malls to lat & long using the OneMaps Search API.

In [15]:
mrt_addresses = mrt_data['mrt_station_english'].tolist()
mall_addresses = mall_data['mall_name'].tolist()
mrt_input, mrt_latitudes, mrt_longitudes, mrt_not_found = getcoordinates(mrt_addresses)
mall_input, mall_latitudes, mall_longitudes, mall_not_found = getcoordinates(mall_addresses)

df_geo_mrt = pd.DataFrame()
df_geo_mrt['address'] = pd.Series(mrt_input)
df_geo_mrt['latitude'] = pd.Series(mrt_latitudes)
df_geo_mrt['longitude'] = pd.Series(mrt_longitudes)
display(df_geo_mrt.head())
print(df_geo_mrt.info())

df_geo_mall = pd.DataFrame()
df_geo_mall['address'] = pd.Series(mall_input)
df_geo_mall['latitude'] = pd.Series(mall_latitudes)
df_geo_mall['longitude'] = pd.Series(mall_longitudes)
display(df_geo_mall.head())
print(df_geo_mall.info())




Fetch initiated for 188 urls.
Ten Mile Junction
{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}
Search took 16.73 s
Fetch completed for 188 urls, 187 urls were located, 1 urls could not be found.
Fetch initiated for 125 urls.
City Vibe
{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}
Clarke Quay Central
{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}
Mandarin Gallery
{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}
Shaw House and Centre
{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}
Search took 11.45 s
Fetch completed for 125 urls, 121 urls were located, 4 urls could not be found.


,address,latitude,longitude
0,Bukit Gombak,1.36923536968527,103.760397157939
1,Dakota,1.30981654427805,103.888843288251
2,Marina Bay,1.28466203613219,103.861005919301
3,Sembawang,1.41961709618173,103.821095254872
4,Tuas Link,1.2698666676481,103.627208060877


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   address    187 non-null    object
 1   latitude   187 non-null    object
 2   longitude  187 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB
None


,address,latitude,longitude
0,Beauty World Centre,1.34241264188642,103.776539385406
1,Anchorpoint,1.28893477974497,103.805607779399
2,600 @ Toa Payoh,1.33403623894465,103.850977706475
3,100 AM,1.27468281482263,103.843488359469
4,Causeway Point,1.43622867995805,103.786349823084


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   address    121 non-null    object
 1   latitude   121 non-null    object
 2   longitude  121 non-null    object
dtypes: object(3)
memory usage: 3.0+ KB
None


**Convert lat long to KM wrt CBD**

In [16]:
from geopy import distance
import warnings
warnings.filterwarnings('ignore')
import numpy as np

def latlong2km(df_geo,df_geo_mrt):
    df_out = df_geo.copy()
    cbd_lat = df_geo_mrt[df_geo_mrt['address']=='Raffles Place']['latitude'].iloc[0]
    cbd_lon = df_geo_mrt[df_geo_mrt['address']=='Raffles Place']['longitude'].iloc[0]

    cbd_x_list = []
    cbd_y_list = []

    for address_index, address_row in df_geo.iterrows():
        x_sign = np.sign(float(address_row['longitude'])-float(cbd_lon))
        y_sign = np.sign(float(address_row['latitude'])-float(cbd_lat))

        cbd_x = x_sign*distance.distance((address_row['latitude'],address_row['longitude']),(address_row['latitude'],cbd_lon)).km
        cbd_y = y_sign*distance.distance((address_row['latitude'],address_row['longitude']),(cbd_lat,address_row['longitude'])).km
        cbd_x_list.append(cbd_x)
        cbd_y_list.append(cbd_y)
    df_out['latitude'] = pd.Series(cbd_y_list)
    df_out['longitude'] = pd.Series(cbd_x_list)
    return df_out

df_dist = latlong2km(df_geo,df_geo_mrt)
mrt_dist = latlong2km(df_geo_mrt,df_geo_mrt)
mall_dist = latlong2km(df_geo_mall,df_geo_mrt)

display(df_dist.head())
# display(mrt_dist.head())
# display(mall_dist.head())

# remove duplicates
mrt_dist = mrt_dist.drop_duplicates().reset_index(drop=True)
mall_dist = mall_dist.drop_duplicates().reset_index(drop=True)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(mrt_dist)
    display(mall_dist)

with open('mrt_data.pickle', 'wb') as f:
    pickle.dump(mrt_dist,f)
with open('mall_data.pickle', 'wb') as f:
    pickle.dump(mall_dist,f)


,address,latitude,longitude
0,624 ANG MO KIO AVE 4,10.760611,-1.229515
1,82 BEDOK NTH RD,5.007962,9.944424
2,666 JLN DAMAI,5.453427,6.349306
3,471 ANG MO KIO AVE 10,8.759679,0.629060
4,441 ANG MO KIO AVE 10,9.045435,0.346988


,address,latitude,longitude
0,Bukit Gombak,9.398,-10.089
1,Dakota,2.827,4.206
2,Marina Bay,0.046,1.108
3,Sembawang,14.969,-3.334
4,Tuas Link,-1.590,-24.912
5,Woodlands,16.512,-5.199
6,Lorong Chuan,7.410,1.243
7,Bukit Batok,6.829,-11.574
8,Newton,3.323,-1.102
9,Sengkang,11.880,5.017


,address,latitude,longitude
0,Beauty World Centre,6.432,-8.292
1,Anchorpoint,0.518,-5.057
2,600 @ Toa Payoh,5.505,-0.008
3,100 AM,-1.058,-0.842
4,Causeway Point,16.805,-7.200
5,Beauty World Plaza,6.364,-8.323
6,Change Alley,-0.014,0.166
7,City Square Mall,3.001,0.621
8,Century Square,7.533,10.323
9,Bedok Point,5.120,10.765


In [17]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(mrt_dist)

               address  latitude  longitude
0         Bukit Gombak     9.398    -10.089
1               Dakota     2.827      4.206
2           Marina Bay     0.046      1.108
3            Sembawang    14.969     -3.334
4            Tuas Link    -1.590    -24.912
5            Woodlands    16.512     -5.199
6         Lorong Chuan     7.410      1.243
7          Bukit Batok     6.829    -11.574
8               Newton     3.323     -1.102
9             Sengkang    11.880      5.017
10           City Hall     0.719      0.080
11          Paya Lebar     7.105      3.510
12           Boon Keng     3.320      2.311
13         Tiong Bahru     0.070     -1.893
14          Gul Circle     3.937    -20.234
15          Ang Mo Kio     8.328      0.583
16             Yew Tee    12.542    -11.672
17           Woodleigh     6.275      2.404
18              Bishan     8.948     -1.642
19              Khatib    15.625     -1.171
20          Queenstown     1.423     -4.736
21            Clementi     4.346

**Merge location to main dataset**

Now that we have the distances for each block, we will merge the distances dataframe back to the main housing_data dataset.

In [18]:
df_geo['longitude'] = df_dist['longitude'].astype('float')
df_geo['latitude'] = df_dist['latitude'].astype('float')

housing_data_dist = housing_data_raw.merge(df_geo, left_on=['address'], right_on = ['address'], how='left').dropna()
print(housing_data_dist.info())
display(housing_data_dist.head())

with open('housing_data.pickle', 'wb') as f:
    pickle.dump(housing_data_dist, f)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62214 entries, 0 to 62213
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   town             62214 non-null  object 
 1   flat_type        62214 non-null  object 
 2   block            62214 non-null  object 
 3   street_name      62214 non-null  object 
 4   floor_area_sqm   62214 non-null  float64
 5   flat_model       62214 non-null  object 
 6   remaining_lease  62214 non-null  float64
 7   resale_price     62214 non-null  float64
 8   year             62214 non-null  int64  
 9   storey           62214 non-null  float64
 10  psm              62214 non-null  float64
 11  address          62214 non-null  object 
 12  latitude         62214 non-null  float64
 13  longitude        62214 non-null  float64
dtypes: float64(7), int64(1), object(6)
memory usage: 7.1+ MB
None


,town,flat_type,block,street_name,floor_area_sqm,flat_model,remaining_lease,resale_price,year,storey,psm,address,latitude,longitude
0,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,45.0,Improved,64.0,211000.0,2021,1.0,4688.888889,170 ANG MO KIO AVE 4,9.924554,-1.626898
1,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,45.0,Improved,64.0,225000.0,2021,7.0,5000.000000,170 ANG MO KIO AVE 4,9.924554,-1.626898
2,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,68.0,New Generation,59.0,260000.0,2021,5.0,3823.529412,331 ANG MO KIO AVE 1,8.609894,-0.031596
3,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,68.0,New Generation,58.0,265000.0,2021,4.0,3897.058824,534 ANG MO KIO AVE 10,9.930939,0.346971
4,ANG MO KIO,3 ROOM,561,ANG MO KIO AVE 10,68.0,New Generation,58.0,265000.0,2021,1.0,3897.058824,561 ANG MO KIO AVE 10,9.546050,0.757224
